In [13]:
import pandas as pd

# 📂 Step 1: 파일 로드
user_df = pd.read_csv("../data/VL_csv/tn_traveller_master_여행객 Master_E_preprocessed.csv")
travel_df = pd.read_csv("../data/VL_csv/tn_travel_여행_E_COST_cleaned.csv")
visit_df = pd.read_csv("../data/VL_csv/tn_visit_area_info_방문지정보_Cleaned_E.csv")
move_df = pd.read_csv("../data/VL_csv/tn_move_his_이동내역_E.csv")

# ✅ Step 2: 여행별 visit_area 목록 추출
travel_to_visits = visit_df.groupby("TRAVEL_ID")["VISIT_AREA_ID"].apply(list).to_dict()

# ✅ Step 3: 이동내역 기반 visit_area 간 엣지 구성
edge_set = set()
for travel_id, group in move_df.groupby("TRAVEL_ID"):
    path = []
    for _, row in group.iterrows():
        sid = row["START_VISIT_AREA_ID"]
        eid = row["END_VISIT_AREA_ID"]
        if pd.notna(sid):
            path = [int(sid)]
        if pd.notna(eid):
            path.append(int(eid))
    for a, b in zip(path[:-1], path[1:]):
        edge_set.add((a, b))  # a → b 이동

# 🔹 visit_area ID 정렬 및 index 매핑
visit_area_ids = sorted(visit_df["VISIT_AREA_ID"].dropna().unique().astype(int))
visit_area_id_to_index = {vid: i for i, vid in enumerate(visit_area_ids)}


In [14]:
# torch import 누락되어 다시 불러오기
import torch

# 이동내역 기반 edge 다시 처리
edge_visit_move = list(edge_set)
edge_visit_move_index = torch.tensor(edge_visit_move, dtype=torch.long).T  # shape [2, num_edges]

{
    "total_users": len(user_df),
    "total_travels": len(travel_df),
    "total_visit_areas": len(visit_area_ids),
    "visit_move_edges": len(edge_visit_move_index[0])
}


{'total_users': 1919,
 'total_travels': 2560,
 'total_visit_areas': 1432,
 'visit_move_edges': 2074}

In [15]:
# 🔹 Step 4: user + travel feature 및 label 벡터 생성
import numpy as np

# ✅ 고정된 user feature 컬럼 (25차원)
user_feature_cols = [
    'GENDER', 'EDU_NM', 'EDU_FNSH_SE', 'MARR_STTS', 'JOB_NM', 'HOUSE_INCOME',
    'TRAVEL_TERM', 'TRAVEL_LIKE_SIDO_1', 'TRAVEL_LIKE_SIDO_2', 'TRAVEL_LIKE_SIDO_3',
    'AGE_GRP', 'FAMILY_MEMB', 'TRAVEL_NUM', 'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
    'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'INCOME'
]

# ✅ 고정된 travel feature 컬럼 (직접 정해준 정보 기준)
travel_feature_cols = [
    'TRAVEL_PURPOSE', 'TRAVEL_START_YMD', 'TRAVEL_END_YMD',
    'LODGOUT_COST', 'ACTIVITY_COST', 'TOTAL_COST'
]

# 🔄 TRAVELER_ID 기준 병합
merged_df = travel_df.merge(user_df, on="TRAVELER_ID", how="inner")


In [16]:
import ast

def parse_travel_purpose(purpose_str):
    try:
        values = ast.literal_eval(purpose_str)
        return float(values[0]) if values else 0.0
    except:
        return 0.0

# TRAVEL_PURPOSE 컬럼 처리
merged_df["TRAVEL_PURPOSE"] = merged_df["TRAVEL_PURPOSE"].apply(parse_travel_purpose)


In [17]:
from datetime import datetime

# ✅ 날짜 차이 계산 함수
def compute_days(start_str, end_str):
    try:
        start = datetime.strptime(str(int(start_str)), "%Y%m%d")
        end = datetime.strptime(str(int(end_str)), "%Y%m%d")
        delta = (end - start).days + 1  # ✅ 최소 하루 보장
        return max(delta, 1)
    except:
        return 1  # ✅ 파싱 실패 시에도 최소 1일
    
    
# 여행 일수 추가 컬럼
merged_df["TRAVEL_DAYS"] = merged_df.apply(
    lambda row: compute_days(row["TRAVEL_START_YMD"], row["TRAVEL_END_YMD"]),
    axis=1
)

# ✅ 다시 travel feature 선택 (날짜 대신 일수로 대체)
travel_feature_cols = [
    'TRAVEL_PURPOSE', 'TRAVEL_DAYS',  # 목적 + 일수
    'LODGOUT_COST', 'ACTIVITY_COST', 'TOTAL_COST'  # 비용
]

# 결측값 채우고 float 변환
merged_df[travel_feature_cols] = merged_df[travel_feature_cols].fillna(0).astype(np.float32)

# 결과 미리 보기
merged_df[travel_feature_cols].head()


,TRAVEL_PURPOSE,TRAVEL_DAYS,LODGOUT_COST,ACTIVITY_COST,TOTAL_COST
0,3.0,1.0,9500.0,0.0,19000.0
1,2.0,1.0,0.0,319340.0,319340.0
2,3.0,1.0,0.0,421160.0,421160.0
3,1.0,1.0,0.0,414220.0,414220.0
4,2.0,1.0,0.0,189000.0,189000.0


In [18]:

# 학습용 샘플 목록
user_features, travel_features, travel_labels = [], [], []
valid_travel_ids = []

for idx, row in merged_df.iterrows():
    travel_id = row["TRAVEL_ID"]
    if travel_id not in travel_to_visits:
        continue
    visit_ids = travel_to_visits[travel_id]
    visit_indices = [visit_area_id_to_index[vid] for vid in visit_ids if vid in visit_area_id_to_index]
    if not visit_indices:
        continue

    u_feat = row[user_feature_cols].values.astype(np.float32)
    t_feat = row[travel_feature_cols].values.astype(np.float32)

    label = np.zeros(len(visit_area_ids), dtype=np.float32)
    label[visit_indices] = 1.0

    user_features.append(u_feat)
    travel_features.append(t_feat)
    travel_labels.append(label)
    valid_travel_ids.append(travel_id)

# numpy to tensor
user_x = torch.tensor(np.stack(user_features))       # [N, 25]
travel_x = torch.tensor(np.stack(travel_features))   # [N, 6]
label_y = torch.tensor(np.stack(travel_labels))      # [N, num_visit_area]

{
    "samples": len(valid_travel_ids),
    "user_x_shape": user_x.shape,
    "travel_x_shape": travel_x.shape,
    "label_y_shape": label_y.shape
}

{'samples': 1919,
 'user_x_shape': torch.Size([1919, 25]),
 'travel_x_shape': torch.Size([1919, 5]),
 'label_y_shape': torch.Size([1919, 1432])}

In [19]:
from make_gnn_dataset import build_dataset

In [20]:
hetero_list, visit_area_id_to_index, edge_visit_move_index = build_dataset()
print(hetero_list[0])


HeteroData(
  user={ x=[1, 25] },
  travel={ x=[1, 5] },
  visit_area={
    x=[1432, 64],
    y=[1432],
  },
  (user, traveled, travel)={ edge_index=[2, 1] },
  (visit_area, move_1, visit_area)={ edge_index=[2, 1987] }
)


In [21]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, SAGEConv

class SafeProjectedGNN(nn.Module):
    def __init__(self, metadata, user_input_dim=25, travel_input_dim=5, hidden_dim=64):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.metadata = metadata

        self.input_proj = nn.ModuleDict({
            'user': nn.Linear(user_input_dim, hidden_dim),
            'travel': nn.Linear(travel_input_dim, hidden_dim),
            'visit_area': nn.Identity()
        })

        self.conv1 = HeteroConv({
            edge_type: SAGEConv((hidden_dim, hidden_dim), hidden_dim)
            for edge_type in metadata[1]
        }, aggr='sum')

        self.conv2 = HeteroConv({
            edge_type: SAGEConv((hidden_dim, hidden_dim), hidden_dim)
            for edge_type in metadata[1]
        }, aggr='sum')

        self.dropout = nn.Dropout(0.3)

        self.scorer = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)
        )

    def forward(self, x_dict, edge_index_dict, feedback_mask=None):
        # 1. input projection
        x_dict = {
            k: self.input_proj[k](v) if k in self.input_proj and v is not None else v
            for k, v in x_dict.items()
        }

        # 2. 안전한 HeteroConv
        h_dict = self.safe_conv_layer(self.conv1, x_dict, edge_index_dict)
        h_dict = self.safe_conv_layer(self.conv2, h_dict, edge_index_dict)

        # 3. visit_area score 추출
        h_visit = h_dict.get('visit_area', None)
        if h_visit is None:
            raise ValueError("visit_area 노드에 대한 표현이 없습니다.")
        score = self.scorer(h_visit).squeeze(-1)

        if feedback_mask is not None:
            score += feedback_mask

        return score

    def safe_conv_layer(self, conv, x_dict, edge_index_dict):
        h_dict = {}
        for edge_type in edge_index_dict:
            if edge_type not in conv.convs:
                continue
            src, _, dst = edge_type
            if src not in x_dict or dst not in x_dict:
                continue
            x_src = x_dict[src]
            x_dst = x_dict[dst]
            if x_src is None or x_dst is None:
                continue
            edge_index = edge_index_dict[edge_type]
            if edge_index is None or edge_index.numel() == 0:
                continue
            try:
                out = conv.convs[edge_type]((x_src, x_dst), edge_index)
                if dst not in h_dict:
                    h_dict[dst] = []
                h_dict[dst].append(out)
            except Exception as e:
                print(f"[Skip edge {edge_type}] due to error: {e}")
                continue
        return {
            k: self.dropout(F.relu(torch.stack(v).sum(0))) if v else None
            for k, v in h_dict.items()
        }


In [22]:
from train_visit_recommender import train_visit_recommender
from make_gnn_dataset import build_dataset

hetero_list, _, _ = build_dataset()
metadata = hetero_list[0].metadata()
model = SafeProjectedGNN(metadata=metadata, user_input_dim=25, travel_input_dim=5)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_visit_recommender(model, hetero_list, optimizer, num_epochs=10)

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/torch_geometric/nn/conv/hetero_conv.py:76: UserWarning: There exist node types ({'user'}) whose representations do not get updated during message passing as they do not occur as destination type in any edge type. This may lead to unexpected behavior.
  warnings.warn(


[Epoch 1] Loss: 0.0317
[Epoch 2] Loss: 0.0277
[Epoch 3] Loss: 0.0275
[Epoch 4] Loss: 0.0275
[Epoch 5] Loss: 0.0274
[Epoch 6] Loss: 0.0274
[Epoch 7] Loss: 0.0274
[Epoch 8] Loss: 0.0274
[Epoch 9] Loss: 0.0274
[Epoch 10] Loss: 0.0274


In [47]:
from recommend_and_decode_v2 import recommend_and_decode

# 예시 입력
user_input = user_x[0].tolist()       # [25]
travel_input = travel_x[0].tolist()   # [5]

# 역 매핑
index_to_visit_area_id = {v: k for k, v in visit_area_id_to_index.items()}

result_df = recommend_and_decode(
    model=model,
    user_feat=user_x[0].tolist(),
    travel_feat=travel_x[0].tolist(),
    visit_area_ids=list(visit_area_id_to_index.keys()),
    index_to_visit_area_id={v: k for k, v in visit_area_id_to_index.items()},
    visit_move_edge_index=edge_visit_move_index,
    visit_df=visit_df,
    hidden_dim=64,
    top_k=20
)

print(result_df)


    VISIT_AREA_ID VISIT_AREA_NM     SCORE
0      2307090001    저스트 단동 녹번점 -5.353099
1      2307020001           노들섬 -5.353099
2      2307010001        연천 감자탕 -5.353099
3      2306180001           동구릉 -5.353099
4      2306050001      초계국수 칼국수 -5.353099
5      2306040001         쉬자 파크 -5.353099
6      2305290001  토요코인 호텔 영등포점 -5.353099
7      2305280001       덕수궁 중명전 -5.353099
8      2305210001   두부 마당 포곡 본점 -5.353099
9      2304290003       스타필드 안성 -5.353100
10     2304290002       농협안성팜랜드 -5.353100
11     2304280006       효자손 왕만두 -5.353100
12     2304280005   해브펀 캠핑 어라운드 -5.353100
13     2304280003         삼환아파트 -5.353100
14     2304290006    충무아트센터 대극장 -5.353100
15     2304300001       평택 초등학교 -5.353100
16     2304290007        호텔 프라하 -5.353100
17     2304290005            편지 -5.353100
18     2304290004     대동국수 평택역점 -5.353100
19     2304280007           숭례문 -5.353100


In [48]:
visit_df.columns

Index(['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM',
       'VISIT_START_YMD', 'VISIT_END_YMD', 'ROAD_NM_ADDR', 'LOTNO_ADDR',
       'X_COORD', 'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD', 'POI_ID', 'POI_NM',
       'RESIDENCE_TIME_MIN', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN',
       'VISIT_CHC_REASON_CD', 'LODGING_TYPE_CD', 'DGSTFN', 'REVISIT_INTENTION',
       'RCMDTN_INTENTION', 'SGG_CD'],
      dtype='object')

In [49]:
result_merged = result_df.merge(
    visit_df[['VISIT_AREA_NM', 'ROAD_NM_ADDR', 'LOTNO_ADDR', 'X_COORD', 'Y_COORD']].drop_duplicates(subset='VISIT_AREA_NM'),
    on="VISIT_AREA_NM",
    how="left"
).dropna(subset='X_COORD')


In [50]:
result_merged

,VISIT_AREA_ID,VISIT_AREA_NM,SCORE,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD
0,2307090001,저스트 단동 녹번점,-5.353099,서울 은평구 은평로 220,서울 은평구 응암동 769,126.930556,37.599462
1,2307020001,노들섬,-5.353099,서울 용산구 양녕로 445,서울 용산구 이촌동 302-146,126.958034,37.517660
2,2307010001,연천 감자탕,-5.353099,경기 연천군 연천읍 문화로 133,경기 연천군 연천읍 현가리 73-13,127.078592,38.105661
3,2306180001,동구릉,-5.353099,경기 구리시 동구릉로 197,경기 구리시 인창동 66,127.132050,37.619415
4,2306050001,초계국수 칼국수,-5.353099,경기 여주시 신륵사길 6-27,경기 여주시 천송동 289-7,127.653838,37.298523
6,2305290001,토요코인 호텔 영등포점,-5.353099,서울 영등포구 신길로 293,서울 영등포구 영등포동1가 106,126.912013,37.517622
7,2305280001,덕수궁 중명전,-5.353099,서울 중구 정동길 41-11,서울 중구 정동 1-11,126.972522,37.566735
8,2305210001,두부 마당 포곡 본점,-5.353099,경기 용인시 처인구 포곡읍 포곡로 339,경기 용인시 처인구 포곡읍 삼계리 401-3,127.232926,37.284580
9,2304290003,스타필드 안성,-5.353100,경기 안성시 공도읍 서동대로 3930-39,경기 안성시 공도읍 진사리 354,127.147095,36.995025
10,2304290002,농협안성팜랜드,-5.353100,경기 안성시 공도읍 대신두길 28,경기 안성시 공도읍 신두리 451,127.193517,36.991317


In [51]:
import folium

def visualize_recommendations_on_map(result_df):
    # 중심 좌표는 첫 번째 추천 결과 기준
    center_lat = result_df.iloc[0]['Y_COORD']
    center_lon = result_df.iloc[0]['X_COORD']

    m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

    for _, row in result_df.iterrows():
        lat, lon = row['Y_COORD'], row['X_COORD']
        name = row['VISIT_AREA_NM']
        score = row.get('SCORE', None)
        popup_text = f"{name}<br>Score: {score:.2f}" if score else name

        folium.Marker(
            location=[lat, lon],
            popup=popup_text,
            tooltip=name,
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(m)

    return m


In [52]:
map_obj = visualize_recommendations_on_map(result_merged)
map_obj.save("recommend_map.html")
map_obj  # Jupyter에서는 이 줄만 있으면 바로 표시됨

### todo list
- 방문지 정보 정리 (집, 아파트, 호텔)
- 이동정보 데이터도 정리해야됨
- 방문지 및 이동정보 정리한 것 기반으로 리펙토링 할 것